# Functional Dependency

In this exercise we will look at how different functionals perform for different kind of systems. We will be looking at the following functionals (these are also their names in GPAW. Note that upper and lower case matters!):

* LDA
* PBE
* RPBE
 * Optimized PBE for catalysis
* PBEsol
 * A varation of PBE optimized for solids
* BEEF-vdW

We will compare 3 different systems:
* Benzene
* NaCl
* Graphite

Before we start, try and think about which functionals you would expect to perform the best for each system.

### Experimental reference distances

Benzene: C-C distance: $1.39$ Å

https://chem.libretexts.org/Textbook_Maps/Organic_Chemistry/Map%3A_Organic_Chemistry_(McMurry)/Chapter_15%3A_Benzene_and_Aromaticity/15.02_Structure_and_Stability_of_Benzene

NaCl: $a=c=5.640$ Å

http://www.ilpi.com/inorganic/structures/nacl/index.html

Graphite: $a=2.46$ Å and $c=6.71$ Å

http://www.phy.ohiou.edu/~asmith/NewATOMS/HOPG.pdf

### Questions
As you will need to do cell optimizations, you will need to do plane-wave calculations, as this is the only mode in which you can get the stress tensor. Use the UnitCellFilter to relax cell parameters. A $500 \ \text{eV}$ plane-wave cutoff will be sufficient for this exercise.

* When do we need to include k-points?
* Compare DFT results with the experimental references
* How do the results compare to your initial guesses?
* Which functional performs best for each system? 

* Do any of the functionals get any of the systems completely wrong?

In [7]:
%%writefile C6H6.py
from ase.build import molecule
from gpaw import GPAW, PW
from ase.optimize import BFGS

for fxc in ['LDA','PBE','RPBE','PBEsol','BEEF-vdW']:
    atoms = molecule('C6H6')
    atoms.center(vacuum=7)
    calc = GPAW(xc=fxc,mode=PW(500),txt='C6H6_%s.txt' %fxc)
    atoms.set_calculator(calc)
    dyn = BFGS(atoms, trajectory='C6H6_%s.traj' %fxc)
    dyn.run(0.03)

Writing C6H6.py


In [1]:
!qsub.py -p 4 -t 2 C6H6.py

7246940.hnode2


In [3]:
!qstat

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
7245977.hnode2             C6H6.py          s171629         04:38:08 C hpc            
7246940.hnode2             C6H6.py          s171629                0 R hpc            


In [10]:
%%writefile NaCl.py
from ase.constraints import StrainFilter
from ase.spacegroup import crystal
from gpaw import GPAW, PW
from ase.optimize import BFGS

for fxc in ['LDA','PBE','RPBE','PBEsol','BEEF-vdW']:
    a = 5.6  
    atoms = crystal(['Na', 'Cl'], [(0, 0, 0), (0.5, 0.5, 0.5)], spacegroup=225, cellpar=[a, a, a, 90, 90, 90])
    calc = GPAW(xc=fxc,mode=PW(500),kpts=(4, 4, 4),txt='NaCl_%s.txt' %fxc)
    atoms.set_calculator(calc)
    sf = StrainFilter(atoms)
    dyn = BFGS(sf, trajectory='NaCl_%s.traj' %fxc)
    dyn.run(0.03)

Writing NaCl.py


In [11]:
!qsub.py -p 4 -t 1 NaCl.py

7245978.hnode2


In [12]:
!qstat

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
7245965.hnode2             C6H6.py          s171629         01:06:02 C hpc            
7245977.hnode2             C6H6.py          s171629                0 R hpc            
7245978.hnode2             NaCl.py          s171629                0 R hpc            


In [13]:
%%writefile Graphite.py
from ase.lattice.hexagonal import Graphite
from ase.constraints import StrainFilter
from gpaw import GPAW, PW
from ase.optimize import BFGS
from ase.visualize import view

for fxc in ['LDA','PBE','RPBE','PBEsol','BEEF-vdW']:
    atoms = Graphite('C', latticeconstant={'a': 2.5, 'c': 6.5})
    view(atoms, repeat=(4, 4 , 2))
    calc = GPAW(xc=fxc,mode=PW(500),kpts=(4, 4, 4),txt='Graphite_%s.txt' %fxc)
    atoms.set_calculator(calc)
    sf = StrainFilter(atoms)
    dyn = BFGS(sf, trajectory='Graphite_%s.traj' %fxc)
    dyn.run(0.03)

Writing Graphite.py


In [14]:
!qsub.py -p 4 -t 1 Graphite.py

7245979.hnode2


In [8]:
!qstat

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
7246940.hnode2             C6H6.py          s171629         03:53:37 R hpc            


In [4]:
!showq


active jobs------------------------
JOBID              USERNAME      STATE PROCS   REMAINING            STARTTIME

7246733             s134816    Running    48    00:10:41  Thu Sep 20 15:06:20
7246895                pkam    Running     1    00:21:41  Thu Sep 20 15:22:20
7246896                pkam    Running     1    00:21:43  Thu Sep 20 15:22:22
7246924                pkam    Running     1    00:21:49  Thu Sep 20 15:22:28
7245973                mtor    Running     8    00:57:35  Thu Sep 20 13:53:14
7239005               giale    Running    32     1:32:50  Mon Sep 17 16:58:29
7246940             s171629    Running     4     1:59:45  Thu Sep 20 15:25:24
7244803             s134142    Running    32     2:03:47  Thu Sep 20 09:29:26
7239106               giale    Running    32     3:11:30  Mon Sep 17 18:37:09
7245966                yiyu    Running    18     3:23:31  Thu Sep 20 13:49:10
7245968                yiyu    Running    20     3:24:24  Thu Sep 20 13:50:03
7245969                yiy